In [ ]:
import pandas as pd
import numpy as np
from datetime import date
import requests
import warnings

from funciones import *
warnings.filterwarnings("ignore")
desde=str(date.today())


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

url_1="https://api.bcra.gob.ar/estadisticas/v4.0/monetarias"
json=requests.get(url_1,verify=False).json()
result=json.get("results")

info_var=[]
for i in result:
    if i.get("categoria") == "Principales Variables": 
        diccionario={"idVariable":i.get("idVariable"),"Descripcion":i.get("descripcion"),"primer_fecha":i.get("primerFechaInformada"),"ultima_fecha":i.get("ultFechaInformada")}
        info_var.append(diccionario)
    else:
        pass

lista_df=[]
for i in range(0,len(info_var)):
    lista_pag=[]
    offset=0
    limit=1000
    desde="2003-12-10"# FECHA
    while salir is False: 
        #busco info
        url_2=f'{url_1}/{info_var[i].get("idVariable")}?hasta={info_var[i].get("ultima_fecha")}&desde={desde}&offset={offset}&limit={limit}'
        try:
            conex=requests.get(url_2,verify=False)
            if conex.status_code==200:
                json=conex.json()
                info=json.get("results")
                if info is not None:
                    detalle=info[0].get("detalle")
                    #paginado
                    lista_pag.extend(detalle)
                    total=json.get("metadata").get("resultset").get("count")
                    offset+=limit

                    if offset>=total:
                        df=pd.DataFrame(lista_pag)
                        salir=True

                elif info is None and len(lista_pag)!=0:#por si una variable solo se procesa parcialmente, la 24 lo necesitaba para fechas previas al 2003-01-02
                    df = pd.DataFrame(lista_pag)
                    salir=True

                elif info is None and len(lista_pag)==0:
                    salir=True 
            else:#por si falla la conexion con alguna variable(TAMAR EN diciembre y principio de enero)
                salir=True
        except Exception as e:
                
                salir = True      
    #uno resultados mantener if porque se rompe fácil la api
    if len(lista_pag) > 0:
            df = pd.DataFrame(lista_pag)
            if 'fecha' in df.columns:
                df["fecha"]=pd.to_datetime(df["fecha"])
                df=df.set_index("fecha")
                df=df.rename(columns={"valor":f"{info_var[i].get("Descripcion")}"})
                lista_df.append(df)
            else:
               print() #nada

df_total=pd.concat(lista_df,join="outer",axis=1)
df_total=df_total.loc[:,~df_total.columns.duplicated()]
df_total=df_total.reset_index()

presidentes=[{"Presidente":"Javier Milei","inicio":"2023-12-10","fin":"2027-12-9"},
             {"Presidente":"Alberto Fernandez","inicio":"2019-12-10","fin":"2023-12-9"},
             {"Presidente":"Mauricio Macri","inicio":"2015-12-10","fin":"2019-12-9"},
             {"Presidente":"Cristina Kirchner","inicio":"2011-12-10","fin":"2015-12-9"},
             {"Presidente":"Cristina Kirchner","inicio":"2007-12-10","fin":"2011-12-9"},
             {"Presidente":"Nestor Kirchner","inicio":"2003-12-10","fin":"2007-12-9"},
             ]

eventos=[{"evento":"","fecha":"","descripción":"","comentario":""}]

df_total["presidente"]=np.nan
for i in presidentes:
    condiciones=((df_total["fecha"]>= i.get("inicio"))&(df_total["fecha"]<=i.get("fin")))
    df_total["presidente"]=np.where(condiciones,i.get("Presidente"),df_total["presidente"])

df_presidencias=pd.DataFrame(presidentes)
df_info_var=pd.DataFrame(info_var)



In [96]:
df_total

,fecha,Reservas internacionales,Tipo de cambio minorista (promedio vendedor),Tipo de cambio mayorista de referencia,Tasa de interés BADLAR de bancos privados,Tasa de interés TM20 de bancos privados,Tasa de interés de pases pasivos para el BCRA a 1 día de plazo,Tasa de interés de préstamos entre entidades financiera privadas (BAIBAR),Tasa de interés de depósitos a 30 días de plazo en entidades financieras,Tasa de interés por adelantos en cuenta corriente,...,Variación interanual del índice de precios al consumidor,Mediana de la variación interanual próximos 12 meses del índice de precios al consumidor del relevamiento de expectativas de mercado,Coeficiente de estabilización de referencia (base 2.2.02=1),Unidad de valor adquisitivo (base 31.3.16=14.05),Unidad de vivienda (base 31.3.16=14.05),Índice para Contratos de Locación (base 30.6.20=1),Serie de tasa de interes para uso de la justicia dispuesta por comunicado P 14.290 (base 01/04/1991),Tasa de interes TAMAR de bancos privados,Tasa de interes TAMAR de bancos públicos,presidente
0,2003-12-10,13917.0,NaN,2.94,2.9375,NaN,NaN,2.05,3.94,NaN,...,NaN,NaN,1.4544,NaN,NaN,NaN,252.3146,NaN,NaN,Nestor Kirchner
1,2003-12-11,13938.0,NaN,2.95,2.3125,NaN,NaN,1.97,2.75,NaN,...,NaN,NaN,1.4545,NaN,NaN,NaN,252.3474,NaN,NaN,Nestor Kirchner
2,2003-12-12,13976.0,NaN,2.96,2.1250,NaN,NaN,1.92,3.89,NaN,...,NaN,NaN,1.4546,NaN,NaN,NaN,252.3802,NaN,NaN,Nestor Kirchner
3,2003-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.4547,NaN,NaN,NaN,252.4048,NaN,NaN,Nestor Kirchner
4,2003-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.4548,NaN,NaN,NaN,252.4294,NaN,NaN,Nestor Kirchner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8067,2026-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,682.2177,1721.45,1272.68,29.61,23709.8405,NaN,NaN,Javier Milei
8068,2026-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,682.7613,1721.45,1272.68,29.63,23724.5494,NaN,NaN,Javier Milei
8069,2026-01-12,44766.0,1482.33,1466.65,27.3750,28.6875,NaN,35.50,25.86,41.59,...,NaN,NaN,683.3054,1721.45,1272.68,29.65,23739.2674,32.4375,37.69,Javier Milei
8070,2026-01-13,NaN,1482.52,1463.32,29.2500,30.8125,NaN,47.84,28.56,49.99,...,NaN,NaN,683.8499,1725.57,1276.08,29.68,23753.9945,34.4375,40.39,Javier Milei


In [99]:
crear_tabla_sql("localhost","variables_por_presidente",df_total,"variables_principales")
crear_tabla_sql("localhost","variables_por_presidente",df_presidencias,"presidencias")
crear_tabla_sql("localhost","variables_por_presidente",df_info_var,"info_variables")
crear_tabla_sql("localhost","variables_por_presidente",df_indice,"indice")